In [60]:
import config
import pandas as pd
import numpy as np
import re
from multiprocessing import cpu_count, Pool

In [61]:
# Load movie conversations and extract the series of utterances
conversations_DF = pd.read_csv(config.movie_conversations_filename,  sep='\+\+\+\$\+\+\+', header=None, engine='python', mangle_dupe_cols=True, verbose=True, names=['characterID_1', 'characterID_2', 'movieID', 'listofutterances'])
conversations_DF['listofutterances'] = conversations_DF['listofutterances'].apply(lambda x: x.replace(']','').replace('[','').replace(' ', '').replace("'", "").replace('"',''))

In [62]:
# Load movie lines into a dataframe and perform some preprocessing on the textual data
movie_lines_DF = pd.read_csv(config.movie_lines_filename,  sep='\+\+\+\$\+\+\+', header=None, engine='python', mangle_dupe_cols=True, verbose=True, names=['lineID', 'characterID', 'movieID', 'charactername', 'text'])
movie_lines_DF['lineID'] = movie_lines_DF['lineID'].apply(lambda x: x.strip())
movie_lines_DF['text'] = movie_lines_DF['text'].apply(lambda x: re.sub('([\w]+)([,;.?!#&-\'\"-]+)([\w]+)?', r'\1 \2 \3', str(x)))
movie_lines_DF['text'] = movie_lines_DF['text'].apply(lambda x: re.sub('[^A-Za-z0-9 ]', '', str(x)))
movie_lines_DF['text'] = movie_lines_DF['text'].apply(lambda x: str(x).strip())
movie_lines_DF['text'] = movie_lines_DF['text'].apply(lambda x: re.sub('  +', ' ', str(x)))

Filled 267 NA values in column text


In [66]:
# Form question answer pairs based on list of utterances from movies lines dataframe 
question_arr = []
answers_arr = []
for idx, row in conversations_DF.iterrows():
    if idx == 10000:
        break
    else:    
        sequence = row[-1].split(",")
        for question_line, answer_line in zip(sequence[:], sequence[1:]):
            question_arr.append(movie_lines_DF.loc[movie_lines_DF['lineID'] == question_line]['text'].values[0])
            answers_arr.append(movie_lines_DF.loc[movie_lines_DF['lineID'] == answer_line]['text'].values[0])        

In [67]:
question_arr = np.array(question_arr)
answers_arr = np.array(answers_arr)
print(question_arr.shape)
print(answers_arr.shape)

(25149,)
(25149,)


In [68]:
# Load the dataset into a pickle file
import pickle

with open("data.pickle", "wb") as f:
    pickle.dump((question_arr,answers_arr), f)

In [69]:
# loading the data for display
with open("data.pickle", "rb") as f:
    quests, ans = pickle.load(f) 
    print(quests)

['Can we make this quick Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break up on the quad Again'
 'Well I thought we d start with pronunciation if that s okay with you'
 'Not the hacking and gagging and spitting part Please' ...
 'Well lose it And why aren t you pushing the large sizes Didn t you get training about upsizing'
 'But I feel weird it s so sleazy' 'It s not optional']
